# NO2 Pollution Barcelona

Air Quality info: https://www.airnow.gov/sites/default/files/2018-06/no2.pdf

In [1]:
# Importing packages and data

import pandas as pd
import re

df = pd.read_csv('../datasets/2.-Urban-Environment/air-quality-nov-2017.csv')

df.head(3)

,Station,Air Quality,Longitude,Latitude,O3 Hour,O3 Quality,O3 Value,NO2 Hour,NO2 Quality,NO2 Value,PM10 Hour,PM10 Quality,PM10 Value,Generated,Date Time
0,Barcelona - Sants,Good,2.1331,41.3788,NaN,NaN,NaN,0h,Good,84.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
1,Barcelona - Eixample,Moderate,2.1538,41.3853,0h,Good,1.0,0h,Moderate,113.0,0h,Good,36.0,01/11/2018 0:00,1541027104
2,Barcelona - Gràcia,Good,2.1534,41.3987,0h,Good,10.0,0h,Good,73.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104


In [2]:
#creating no2 df and renaming columns

df_no2 = df[['Station', 'Air Quality', 'Longitude', 'Latitude', 'NO2 Hour', 'NO2 Quality', 'NO2 Value', 'Generated', 'Date Time']].copy()
df_no2.columns = ['station', 'air_quality', 'longitude', 'latitude', 'no2_hour', 'no2_quality', 'no2_value', 'generated', 'date_time']

**Min, Max, Mean, Median NO2 Value by station:**

In [3]:
df_no2.groupby('station').no2_value.agg(['min', 'max', 'mean'])

,min,max,mean
station,,,
Barcelona - Ciutadella,5.0,88.0,39.855920
Barcelona - Eixample,17.0,113.0,56.240683
Barcelona - Gràcia,6.0,117.0,44.574534
Barcelona - Observ Fabra,1.0,71.0,11.449213
Barcelona - Palau Reial,1.0,99.0,27.974322
Barcelona - Poblenou,5.0,105.0,41.161103
Barcelona - Sants,7.0,89.0,36.359165
Barcelona - Vall Hebron,6.0,91.0,30.812940


**The overall highest values:**

In [4]:
df_no2_short = df_no2[['station', 'no2_hour', 'no2_quality', 'no2_value']].copy()

In [5]:
df_no2_short.set_index('station', inplace=True)
largest = df_no2_short.nlargest(columns='no2_value', n = 10)
largest.columns = ['NO2 Time', 'NO2 Quality', 'NO2 Value']
largest

,NO2 Time,NO2 Quality,NO2 Value
station,,,
Barcelona - Gràcia,10h,Moderate,117.0
Barcelona - Gràcia,9h,Moderate,116.0
Barcelona - Gràcia,9h,Moderate,115.0
Barcelona - Eixample,0h,Moderate,113.0
Barcelona - Eixample,9h,Moderate,112.0
Barcelona - Eixample,11h,Moderate,109.0
Barcelona - Eixample,10h,Moderate,108.0
Barcelona - Gràcia,11h,Moderate,107.0
Barcelona - Eixample,10h,Moderate,105.0


**Hourly NO2 data**

In [8]:
# Dropping missing values

nantodrop = df_no2[df_no2['no2_value'].isna() == True].index.tolist()
df_no2 = df_no2.drop(nantodrop)
df_no2.reset_index(drop=True,inplace=True)

df_no2.head(3)

,station,air_quality,longitude,latitude,no2_hour,no2_quality,no2_value,generated,date_time
0,Barcelona - Sants,Good,2.1331,41.3788,0h,Good,84.0,01/11/2018 0:00,1541027104
1,Barcelona - Eixample,Moderate,2.1538,41.3853,0h,Moderate,113.0,01/11/2018 0:00,1541027104
2,Barcelona - Gràcia,Good,2.1534,41.3987,0h,Good,73.0,01/11/2018 0:00,1541027104


In [9]:
#Removing "h" from Hour column elements in a new column

df_no2['time'] = list(map(lambda x: int(re.findall(r"\d+",x)[0]),df_no2['no2_hour']))
df_no2.rename(columns={'time':'Time'},inplace=True)
df_no2.drop(columns=['no2_hour'],inplace=True)
df_no2.head(3)

,station,air_quality,longitude,latitude,no2_quality,no2_value,generated,date_time,Time
0,Barcelona - Sants,Good,2.1331,41.3788,Good,84.0,01/11/2018 0:00,1541027104,0
1,Barcelona - Eixample,Moderate,2.1538,41.3853,Moderate,113.0,01/11/2018 0:00,1541027104,0
2,Barcelona - Gràcia,Good,2.1534,41.3987,Good,73.0,01/11/2018 0:00,1541027104,0


In [10]:
#Getting Hourly Data

hourly = df_no2.groupby('Time').no2_value.agg(['min', 'max', 'mean'])
hourly

,min,max,mean
Time,,,
0,2.0,113.0,42.854890
1,2.0,96.0,35.714286
2,2.0,86.0,30.235808
3,1.0,77.0,26.772926
4,1.0,65.0,24.644444
5,1.0,67.0,23.298701
6,1.0,64.0,24.314894
7,1.0,80.0,30.372294
8,1.0,103.0,40.004237


In [12]:
# Splitting Dataframe

hourly_df1 = hourly.iloc[:12,:]
hourly_df2 = hourly.iloc[12:,:]
hourly_df1

,min,max,mean
Time,,,
0,2.0,113.0,42.854890
1,2.0,96.0,35.714286
2,2.0,86.0,30.235808
3,1.0,77.0,26.772926
4,1.0,65.0,24.644444
5,1.0,67.0,23.298701
6,1.0,64.0,24.314894
7,1.0,80.0,30.372294
8,1.0,103.0,40.004237


In [10]:
hourly.T

Time,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
min,2.00000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,3.000000,3.000000,6.000000,6.00,4.000000,4.000000,3.000000,3.000000
max,113.00000,96.000000,86.000000,77.000000,65.000000,67.000000,64.000000,80.000000,103.000000,116.000000,...,71.000000,88.000000,84.000000,80.000000,89.000000,101.00,105.000000,97.000000,97.000000,88.000000
mean,42.85489,35.714286,30.235808,26.772926,24.644444,23.298701,24.314894,30.372294,40.004237,46.268908,...,28.053398,28.342723,28.910714,32.728111,42.253219,49.32,50.504505,48.506726,44.532967,39.039735


**Data by Weekday:**

In [36]:
df_mon = df_no2[df_no2['generated'].str.contains('05/11/2018|12/11/2018|19/11/2018|26/11/2018')]
df_tue = df_no2[df_no2['generated'].str.contains('06/11/2018|13/11/2018|20/11/2018|27/11/2018')]
df_wed = df_no2[df_no2['generated'].str.contains('07/11/2018|14/11/2018|21/11/2018|28/11/2018')]
df_thu = df_no2[df_no2['generated'].str.contains('01/11/2018|08/11/2018|15/11/2018|22/11/2018|29/11/2018')]
df_fr = df_no2[df_no2['generated'].str.contains('02/11/2018|09/11/2018|16/11/2018|23/11/2018|30/11/2018')]
df_sa = df_no2[df_no2['generated'].str.contains('03/11/2018|10/11/2018|17/11/2018|24/11/2018')]
df_sun = df_no2[df_no2['generated'].str.contains('04/11/2018|11/11/2018|18/11/2018|25/11/2018')]

In [37]:
df_sun.groupby('station').no2_value.agg(['min', 'max', 'mean', 'median'])

,min,max,mean,median
station,,,,
Barcelona - Ciutadella,5.0,72.0,34.765957,35.0
Barcelona - Eixample,18.0,99.0,44.011494,42.0
Barcelona - Gràcia,6.0,83.0,30.602273,26.5
Barcelona - Observ Fabra,1.0,24.0,7.852632,7.0
Barcelona - Palau Reial,2.0,80.0,16.808511,12.5
Barcelona - Poblenou,6.0,86.0,30.648936,26.5
Barcelona - Sants,9.0,83.0,24.929412,22.0
Barcelona - Vall Hebron,7.0,82.0,17.670213,13.5


In [32]:
mon_mean = df_mon['no2_value'].mean()
tue_mean = df_tue['no2_value'].mean()
wed_mean = df_wed['no2_value'].mean()
thu_mean = df_thu['no2_value'].mean()
fr_mean = df_fr['no2_value'].mean()
sa_mean = df_sa['no2_value'].mean()
sun_mean = df_sun['no2_value'].mean()

In [109]:
weekdays_list = [['Monday', mon_mean], ['Tuesday', tue_mean], ['Wednesday', wed_mean], ['Thursday', thu_mean], ['Friday', fr_mean], ['Saturday', sa_mean], ['Sunday', sun_mean]] 
weekdays = pd.DataFrame(weekdays_list, columns = ['Weekday', 'NO2 Value Mean'])

In [110]:
weekdays.set_index('Weekday', inplace=True)

In [111]:
weekdays

,NO2 Value Mean
Weekday,
Monday,29.031335
Tuesday,31.291951
Wednesday,38.773427
Thursday,45.619699
Friday,44.128767
Saturday,29.661664
Sunday,25.686731


In [95]:
#Separate into one dataframe per week
week_1 = df_no2[df_no2['generated'].str.contains('01/11/2018|02/11/2018|03/11/2018|04/11/2018|05/11/2018|06/11/2018|07/11/2018')]
week_2 = df_no2[df_no2['generated'].str.contains('08/11/2018|09/11/2018|10/11/2018|11/11/2018|12/11/2018|13/11/2018|14/11/2018')]
week_3 = df_no2[df_no2['generated'].str.contains('15/11/2018|16/11/2018|17/11/2018|18/11/2018|19/11/2018|20/11/2018|21/11/2018')]
week_4 = df_no2[df_no2['generated'].str.contains('22/11/2018|23/11/2018|24/11/2018|25/11/2018|26/11/2018|27/11/2018|28/11/2018')]
week_5 = df_no2[df_no2['generated'].str.contains('29/11/2018|30/11/2018')]

In [96]:
w1_mean = week_1['no2_value'].mean()
w2_mean = week_2['no2_value'].mean()
w3_mean = week_3['no2_value'].mean()
w4_mean = week_4['no2_value'].mean()
w5_mean = week_5['no2_value'].mean()

In [100]:
# creating DataFrame for weeks
weeks = [['Week 1', w1_mean], ['Week 2', w2_mean], ['Week 3', w3_mean], ['Week 4', w4_mean]]

weeksdf = pd.DataFrame(weeks,columns=['Weeks','NO2 Value']).set_index('Weeks')
weeksdf

,NO2 Value
Weeks,
Week 1,34.013503
Week 2,35.436096
Week 3,32.219124
Week 4,36.809077
